In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pickle
import tqdm
from sklearn.metrics import mean_squared_error

In [ ]:
plt.rcParams["figure.figsize"] = [16, 9]
# plt.rcParams["figure.dpi"] = 300
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["axes.titlesize"] = 24
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["font.family"] = "serif"

In [ ]:
DATA_REALS = "../data/ohlc_DowJones"

In [ ]:
DATA_PREDS = "../storage/thesis-gan/abllixzn"

In [ ]:
reals = pd.read_csv(f"{DATA_REALS}/ohlc_DowJones_val.csv").filter(regex="mid_price")

In [ ]:
cols = list(reals.columns.values)
d = dict()
for col in cols:
    d[col] = col.split("_")[2]
reals = reals.rename(columns=d)
reals = reals.iloc[:8490, :]
reals_corr = reals.corr(numeric_only=True)

In [ ]:
mses = list()
for epoch in tqdm.tqdm(range(300)):
    PATH_PICKLE_PRICE = DATA_PREDS + f"/preds_epoch={epoch}-target_price=mid_price-target_volume=None.pickle"
    with open(PATH_PICKLE_PRICE, "rb") as handle:
        pred_prices_dict = pickle.load(handle)
    preds = pd.DataFrame(pred_prices_dict["pred_prices"].squeeze().numpy().T)
    preds.columns = reals.columns
    preds_corr = preds.corr(numeric_only=True)
    mse = mean_squared_error(reals_corr, preds_corr)
    mses.append(mse)

In [ ]:
min_epoch, min_mse = min(zip(range(300), mses), key=lambda x: x[1])
min_epoch, min_mse

In [ ]:
plt.plot(mses)
plt.scatter(min_epoch, min_mse, c="r", label=f"Minimum: Epoch={min_epoch} MSE={round(min_mse, 2)}")
plt.legend(loc="upper center", fontsize=15)
plt.title("Average Correlation Distance - Prices")
plt.xlabel("Epoch")
plt.tight_layout()
# plt.savefig(f"/Users/giuseppemasi/PycharmProjects/thesis-gan/storage/thesis-gan/cross_corr_dist_DowJones.png")
plt.show()
plt.close()

In [ ]:
stock_names = [
    "AAPL",
    "AMGN",
    "AXP",
    "BA",
    "CAT",
    "CRM",
    "CSCO",
    "CVX",
    "DIS",
    "GS",
    "HD",
    "HON",
    "IBM",
    "INTC",
    "JNJ",
    "JPM",
    "KO",
    "MCD",
    "MMM",
    "MRK",
    "MSFT",
    "NKE",
    "PG",
    "TRV",
    "UNH",
    "V",
    "VZ",
    "WBA",
    "WMT",
]

In [ ]:
chosen_epoch = 197

In [ ]:
mses[chosen_epoch]

In [ ]:
PATH_PICKLE_PRICE = DATA_PREDS + f"/preds_epoch={chosen_epoch}-target_price=mid_price-target_volume=None.pickle"
with open(PATH_PICKLE_PRICE, "rb") as handle:
    pred_prices_dict = pickle.load(handle)
preds = pd.DataFrame(pred_prices_dict["pred_prices"].squeeze().numpy().T)
preds.columns = reals.columns

In [ ]:
reals = reals.to_numpy()
preds = preds.to_numpy()

In [ ]:
reals.shape, preds.shape

In [ ]:
history_indexes = np.arange(390)
continuation_indexes = np.arange(390, reals.shape[0])
history_indexes.shape, continuation_indexes.shape

In [ ]:
history = reals[:390, :].T
reals_continuation = reals[390:, :].T
preds_continuation = preds[390:, :].T
history.shape, reals_continuation.shape, preds_continuation.shape

In [ ]:
fig, ax = plt.subplots(5, 6)
legend_elements = [
    Line2D([0], [0], color="C0", lw=2, label="Observed"),
    Line2D([0], [0], color="C1", lw=2, label="Real continuation"),
    Line2D([0], [0], color="C2", lw=2, label="Predicted continuation"),
]

for i in range(5):
    for j in range(6):
        linear_index = i * 6 + j
        ax[i, j].set_xticklabels([])
        ax[i, j].set_yticklabels([])
        if linear_index == 29:
            continue
        ax[i, j].set_title(f"{stock_names[linear_index]}", fontsize=20)
        ax[i, j].plot(
            history_indexes,
            history[linear_index],
            color="C0",
        )
        ax[i, j].plot(
            continuation_indexes,
            reals_continuation[linear_index],
            color="C1",
        )
        ax[i, j].plot(
            continuation_indexes,
            preds_continuation[linear_index],
            color="C2",
        )

fig.suptitle("Prices", fontsize=24, y=1.04)
fig.legend(handles=legend_elements, loc="upper center", ncol=3, fontsize=15, bbox_to_anchor=(0.5, 1))
fig.tight_layout()
# plt.savefig("../storage/thesis-gan/prices_DowJones.png")
plt.show()
plt.close(fig)

# Correlations

In [ ]:
plt.rcParams["font.size"] = 8

In [ ]:
sb.heatmap(reals_corr, cmap="Blues", annot=True, fmt=".2f")
plt.title("Real correlations")
plt.show()
plt.close()

In [ ]:
sb.heatmap(preds_corr, cmap="Blues", annot=True, fmt=".2f")
plt.title("Pred correlations")
plt.show()
plt.close()

In [ ]:
diffs_corr = (reals_corr - preds_corr) / reals_corr
sb.heatmap(diffs_corr, cmap="Blues", annot=True, fmt=".2f")
plt.title("Difference correlations")
plt.show()
plt.close()

## Visual analysis

In [ ]:
couples = [
    ("JPM", "UNH"),
    ("IBM", "HON"),
    ("MMM", "CAT"),
    ("MRK", "KO"),
    ("PG", "KO"),
    ("TRV", "CSCO"),
    ("UNH", "TRV"),
    ("MSFT", "UNH"),
    ("MMM", "UNH"),
    ("CVX", "UNH"),
    ("UNH", "BA"),
    ("VZ", "MCD"),
    ("MRK", "WBA"),
    ("V", "MSFT"),
    ("PG", "VZ"),
    ("AAPL", "JNJ"),
]

In [ ]:
def plot_couples(stock_name_1, stock_name_2):
    fig, axes = plt.subplots(2, 2)
    legend_elements = [
        Line2D([0], [0], color="C0", lw=2, label="Observed"),
        Line2D([0], [0], color="C1", lw=2, label="Real continuation"),
        Line2D([0], [0], color="C2", lw=2, label="Predicted continuation"),
    ]

    stock_index_1, stock_index_2 = stock_names.index(stock_name_1), stock_names.index(stock_name_2)
    real_corr, synthetic_corr = reals_corr[stock_name_1][stock_name_2], preds_corr[stock_name_1][stock_name_2]

    reals1, reals2 = reals[:, stock_index_1], reals[:, stock_index_2]
    history1, history2 = history[stock_index_1], history[stock_index_2]
    reals_continuation1, reals_continuation2 = reals_continuation[stock_index_1], reals_continuation[stock_index_2]
    preds_continuation1, preds_continuation2 = preds_continuation[stock_index_1], preds_continuation[stock_index_2]

    # Titles
    axes[0, 0].set_title(f"{stock_name_1} - Real", fontsize=20)
    axes[0, 1].set_title(f"{stock_name_2} - Real", fontsize=20)
    axes[1, 0].set_title(f"{stock_name_1} - Synthetic", fontsize=20)
    axes[1, 1].set_title(f"{stock_name_2} - Synthetic", fontsize=20)

    # Histories
    axes[0, 0].plot(history_indexes, history1, color="C0")
    axes[0, 1].plot(history_indexes, history2, color="C0")
    axes[1, 0].plot(history_indexes, history1, color="C0")
    axes[1, 1].plot(history_indexes, history2, color="C0")

    # Reals
    axes[0, 0].plot(continuation_indexes, reals_continuation1, color="C1")
    axes[0, 1].plot(continuation_indexes, reals_continuation2, color="C1")

    # Synthetic
    axes[1, 0].plot(continuation_indexes, preds_continuation1, color="C2")
    axes[1, 1].plot(continuation_indexes, preds_continuation2, color="C2")

    # y_lim
    axes[0, 0].set_ylim([min(reals1.min(), preds_continuation1.min()), max(reals1.max(), preds_continuation1.max())])
    axes[1, 0].set_ylim([min(reals1.min(), preds_continuation1.min()), max(reals1.max(), preds_continuation1.max())])
    axes[0, 1].set_ylim([min(reals2.min(), preds_continuation2.min()), max(reals2.max(), preds_continuation2.max())])
    axes[1, 1].set_ylim([min(reals2.min(), preds_continuation2.min()), max(reals2.max(), preds_continuation2.max())])

    for axs in axes:
        for ax in axs:
            ax.set_yticklabels([])
            ax.set_xticklabels([])

    axes[0, 1].set_ylabel(round(real_corr, 2), rotation="horizontal", horizontalalignment="right")
    axes[1, 1].set_ylabel(round(synthetic_corr, 2), rotation="horizontal", horizontalalignment="right")

    fig.legend(handles=legend_elements, loc="upper center", ncol=3, fontsize=15, bbox_to_anchor=(0.5, 0.95))
    fig.suptitle("Prices", fontsize=24, y=0.98)
    fig.tight_layout()
    fig.subplots_adjust(top=0.85)
    plt.savefig(f"../storage/thesis-gan/couples_plots/{stock_name_1}-{stock_name_2}.png")
    # plt.show()
    plt.close(fig)

In [ ]:
for stock_name_1, stock_name_2 in couples:
    plot_couples(stock_name_1, stock_name_2)